# Анализ лояльности аудитории телекоммуникационной компании

Данный проект представляет собой анализ результатов опроса, проводившегося телекоммуникационной компаний среди своих клиентов. 
Клиентам задавался следующий вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
По его результатам высчитывался <b>NPS</b> (Net Promoter Score). <br><br>
Также в базе данных представлена <b>следующая информация</b> о клиентах:
- Количество "дней жизни" клиента.
- Возраст.
- Пол.
- Тип операционной системы.
- Тип устройства.
- Страна проживания.
- Город проживания.
- Возрастной сегмент.
- Сегмент по объёму потребляемого трафика.

База данный представлена в SQLite.

<b>Задачи исследования</b> - изучить распределение клиентов в рамках представленной выборки, сравнить NPS среди разных групп, составить портрет "сторонника" компании. <br><br>
В рамках проекта был написан запрос, с локальной машины подгружена ДБ через `sqlalchemy`, далее ДБ выгружена в формате .csv и подгружена в Tableau, в котором уже создан дашборд, также подготовлена презентация с ключевыми результатами анализа. 

In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db = 'telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [4]:
query = """
WITH 
u AS (SELECT *,
        CASE 
            WHEN lt_day < 366 THEN 'новый'
            ELSE 'старый'
        END AS is_new,
        CASE
            WHEN nps_score BETWEEN 0 AND 6 THEN 'критик'
            WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтрал'
            ELSE 'сторонник'
        END AS nps_group,
        CASE
            WHEN gender_segment =1 THEN 'женский'
            WHEN gender_segment =0 THEN 'мужской'
            ELSE 'другой'
        END AS gender_text
    FROM user),
    
a AS (SELECT age_gr_id,
                title
            FROM age_segment),
            
t AS (SELECT tr_gr_id,
                title
            FROM traffic_segment),

ls AS (SELECT lt_gr_id,
                title
            FROM lifetime_segment)

SELECT u.user_id,
    u.lt_day, 
    u.is_new,
    u.age,
    CAST (u.gender_text AS varchar) AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    location.country,
    location.city,
    a.title AS age_segment,
    t.title AS traffic_segment,
    u.nps_score,
    u.nps_group,
    ls.title AS lifetime_segment
    

FROM u
    JOIN location ON location.location_id=u.location_id
    JOIN a ON a.age_gr_id=u.age_gr_id
    JOIN t ON t.tr_gr_id=u.tr_gr_id
    JOIN ls ON ls.lt_gr_id=u.lt_gr_id
"""

In [9]:
df = pd.read_sql(query, engine)

In [10]:
def fast_review(df):
    print ('Общий вид данных')
    display(df.sample(10, random_state = 0))
    print ('-------------------------')
    print ('Общая информация о данных')
    df.info()
    print ('-------------------------')
    print ('Количество дубликатов: {}'.format(df.duplicated().sum()))

In [11]:
fast_review(df)

Общий вид данных


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,nps_score,nps_group,lifetime_segment
120763,G81LF7,1214,старый,27.0,мужской,IOS,SMARTPHONE,Россия,Чебоксары,03 25-34,14 50-55,3,критик,08 36+
340551,RME1PC,567,старый,38.0,мужской,ANDROID,SMARTPHONE,Россия,Челябинск,04 35-44,05 5-10,10,сторонник,06 13-24
271310,O1AOB9,345,новый,43.0,женский,ANDROID,SMARTPHONE,Россия,Новосибирск,04 35-44,07 15-20,10,сторонник,05 7-12
213291,L0QOVV,4887,старый,30.0,женский,ANDROID,SMARTPHONE,Россия,РостовнаДону,03 25-34,05 5-10,6,критик,08 36+
218788,LAZRT8,1446,старый,35.0,мужской,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,12 40-45,10,сторонник,08 36+
338923,RJCIY2,1031,старый,63.0,мужской,ANDROID,SMARTPHONE,Россия,СанктПетербург,06 55-64,03 0.1-1,1,критик,07 25-36
117204,G1CB8H,5595,старый,39.0,мужской,ANDROID,SMARTPHONE,Россия,Воронеж,04 35-44,11 35-40,5,критик,08 36+
373811,TC4TQG,1212,старый,31.0,мужской,ANDROID,SMARTPHONE,Россия,Ульяновск,03 25-34,04 1-5,10,сторонник,08 36+
57975,CZEMGS,2020,старый,35.0,мужской,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,05 5-10,10,сторонник,08 36+
296922,PCXRY4,239,новый,26.0,мужской,ANDROID,SMARTPHONE,Россия,Курск,03 25-34,09 25-30,8,нейтрал,05 7-12


-------------------------
Общая информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  nps_score         502493 non-null  int64  
 12  nps_group         502493 non-null  object 
 13  lifetime_segment  502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB
-----

Очевидные дубликаты отсутствуют, пропусков нет.

In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Cсылка на дашборд на сайте Tableau Public:

https://public.tableau.com/shared/GSKBD4S26?:display_count=n&:origin=viz_share_link

### Cсылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/Whj-1iEPexXCVQ